In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import pickle as pkl
import re
import ast

In [24]:
# specify the directory you want to search in
directory = 'outputs/0718'

# use glob to match the file pattern '*.out'
Baseline_Files = glob.glob(os.path.join(directory, 'Baseline*.pkl'))
Baseline_Files

['outputs/0718/Baseline_BOMP_300_500_1020_nr_0718_0718-214814.pkl']

In [25]:
def parse_omp_logs_to_dataframe(data):
    lines = data.strip().split("\n")
    trials = []
    for line in lines:
        if "Running trials" in line:
            n, p, m = map(int, re.findall(r'\d+', line))
        elif "Cross validating" in line:
            noise_level = float(re.findall(r'\d+\.\d+', line)[0])
        elif "Trial:" in line:
            parts = line.split()
            trial_number = int(parts[1])
            best_k = int(re.findall(r'\d+', parts[5])[0])
            lowest_error = float(parts[8])
            testing_error = float(parts[11])
            trials.append([n, p, m, noise_level, trial_number, best_k, lowest_error, testing_error])
    
    df = pd.DataFrame(trials, columns=['n', 'p', 'm', 'noise_level', 'trial_number', 'best_k', 'lowest_error', 'testing_error'])
    return df

parse_omp_logs_to_dataframe("outputs/0718/Baseline_BOMP_300_500_1020_nr_0718_0718-214814.pkl")

def parse_omp_logs_to_dataframe(input_text):
    lines = input_text.split("\n")
    current_trial_group = None
    current_noise_level = None
    data = []
    
    for line in lines:
        if "Running trials for n =" in line:
            current_trial_group = tuple(map(int, re.findall(r'\d+', line)))
        elif "Cross validating alpha under noise level:" in line:
            current_noise_level = float(line.split()[-1])
        elif "Trial:" in line and current_trial_group and current_noise_level is not None:
            trial_info = re.findall(r'{.*?}', line)[0]
            trial_info = eval(trial_info)
            trial_info["Trial"] = int(line.split()[1])
            trial_info["Lowest CV Error"] = float(re.findall(r'(?<=Lowest CV Error:  )\d+\.\d+', line)[0])
            trial_info["Training Error"] = float(re.findall(r'(?<=Training Error:  )\d+\.\d+', line)[0])
            trial_info["Testing Error"] = float(re.findall(r'(?<=Testing Error:  )\d+\.\d+', line)[0])
            trial_info["n"] = current_trial_group[0]
            trial_info["p"] = current_trial_group[1]
            trial_info["m"] = current_trial_group[2]
            trial_info["Noise Level"] = current_noise_level
            data.append(trial_info)
    
    df = pd.DataFrame(data)
    return df


file_path = 'outputs/0718/slurm-8021496.out'  # replace with your .out file path
logs_from_one_out = parse_logs_to_dataframe(open(file_path).read())
error_dataframe = logs_from_one_out[['Noise Level', 'Testing Error', 'Training Error', 'n', 'p', 'm']].groupby(['n', 'p', 'm','Noise Level']).mean().reset_index()


,n,p,m,noise_level,trial_number,best_k,lowest_error,testing_error


In [22]:
def parse_logs_to_dataframe(input_text):
    lines = input_text.split("\n")
    current_trial_group = None
    current_noise_level = None
    data = []
    
    for line in lines:
        if "Running trials for n =" in line:
            current_trial_group = tuple(map(int, re.findall(r'\d+', line)))
        elif "Cross validating alpha under noise level:" in line:
            current_noise_level = float(line.split()[-1])
        elif "Trial:" in line and current_trial_group and current_noise_level is not None:
            trial_info = re.findall(r'{.*?}', line)[0]
            trial_info = eval(trial_info)
            trial_info["Trial"] = int(line.split()[1])
            trial_info["Lowest CV Error"] = float(re.findall(r'(?<=Lowest CV Error:  )\d+\.\d+', line)[0])
            trial_info["Training Error"] = float(re.findall(r'(?<=Training Error:  )\d+\.\d+', line)[0])
            trial_info["Testing Error"] = float(re.findall(r'(?<=Testing Error:  )\d+\.\d+', line)[0])
            trial_info["n"] = current_trial_group[0]
            trial_info["p"] = current_trial_group[1]
            trial_info["m"] = current_trial_group[2]
            trial_info["Noise Level"] = current_noise_level
            data.append(trial_info)
    
    df = pd.DataFrame(data)
    return df


file_path = 'outputs/0718/slurm-8021496.out'  # replace with your .out file path
logs_from_one_out = parse_logs_to_dataframe(open(file_path).read())
error_dataframe = logs_from_one_out[['Noise Level', 'Testing Error', 'Training Error', 'n', 'p', 'm']].groupby(['n', 'p', 'm','Noise Level']).mean().reset_index()


In [23]:
error_dataframe

,n,p,m,Noise Level,Testing Error,Training Error
0,300,500,10,0.02,0.001472,0.000432
1,300,500,10,0.04,0.003675,0.001159
2,300,500,10,0.06,0.006592,0.001900
3,300,500,10,0.08,0.010767,0.003482
4,300,500,10,0.10,0.017413,0.005383
5,300,500,20,0.02,0.006332,0.001103
6,300,500,20,0.04,0.007968,0.001370
7,300,500,20,0.06,0.011909,0.001728
8,300,500,20,0.08,0.016361,0.002726
9,300,500,20,0.10,0.021508,0.004331
